In [1]:

import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn import metrics
import pennylane as qml

print("CUDA_VISIBLE_DEVICES =", os.environ.get("CUDA_VISIBLE_DEVICES"))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("PyTorch device:", device)

torch.set_default_dtype(torch.float64)
np.set_printoptions(precision=4, suppress=True)


CUDA_VISIBLE_DEVICES = None
PyTorch device: cuda


In [2]:

CSV_PATH = "QIC2025-EstDat.csv"
if not os.path.exists(CSV_PATH):
    alt = "QIC2025-EstDat (1).csv"
    if os.path.exists(alt):
        CSV_PATH = alt

df = pd.read_csv(CSV_PATH)

TARGET   = "DV"
FEATURES = ["BW","COMED","DOSE","TIME","EVID","MDV","AMT","CMT","DVID"]  # 9 özellik
df = df.dropna(subset=FEATURES + [TARGET]).copy()

print("Satır:", len(df), "| Özellik sayısı:", len(FEATURES))
df.head(3)

Satır: 2820 | Özellik sayısı: 9


,ID,BW,COMED,DOSE,TIME,DV,EVID,MDV,AMT,CMT,DVID
0,1,58,0,0,0,18.6174,0,0,0,3,2
1,1,58,0,0,1,13.7783,0,0,0,3,2
2,1,58,0,0,2,16.5747,0,0,0,3,2


In [3]:

rng = np.random.default_rng(42)
uids = df["ID"].unique().copy()
rng.shuffle(uids)

n_total = len(uids)
n_train = int(0.70 * n_total)
n_val   = int(0.15 * n_total)

train_ids = uids[:n_train]
val_ids   = uids[n_train:n_train+n_val]
test_ids  = uids[n_train+n_val:]

train_df = df[df["ID"].isin(train_ids)].copy()
val_df   = df[df["ID"].isin(val_ids)].copy()
test_df  = df[df["ID"].isin(test_ids)].copy()


print("Train/Val/Test şekilleri:", train_df.shape, val_df.shape, test_df.shape)


Train/Val/Test şekilleri: (2040, 11) (355, 11) (425, 11)


In [4]:

X_train = train_df[FEATURES].to_numpy(dtype=np.float64)
X_val   = val_df[FEATURES].to_numpy(dtype=np.float64)
X_test  = test_df[FEATURES].to_numpy(dtype=np.float64)

y_train = train_df[TARGET].to_numpy(dtype=np.float64)
y_val   = val_df[TARGET].to_numpy(dtype=np.float64)
y_test  = test_df[TARGET].to_numpy(dtype=np.float64)

mean_ = X_train.mean(axis=0, keepdims=True)
std_  = X_train.std(axis=0, keepdims=True)
std_[std_ == 0] = 1.0
eps = 1e-8

def zscore(x):
    return (x - mean_) / (std_ + eps)

TrainX = zscore(X_train).astype(np.float64)
ValX   = zscore(X_val).astype(np.float64)
TestX  = zscore(X_test).astype(np.float64)

def pad_numpy(x_np, target_len=16):
    x_np = np.asarray(x_np, dtype=np.float64)
    cur = x_np.shape[1]
    if cur >= target_len:
        return x_np[:, :target_len]
    pad = np.zeros((x_np.shape[0], target_len - cur), dtype=np.float64)
    return np.concatenate([x_np, pad], axis=1)

TrainX_p = pad_numpy(TrainX, 16)
ValX_p   = pad_numpy(ValX, 16)
TestX_p  = pad_numpy(TestX, 16)

print("Padded shapes:", TrainX_p.shape, ValX_p.shape, TestX_p.shape)

Padded shapes: (2040, 16) (355, 16) (425, 16)


In [5]:

Xtr = torch.from_numpy(TrainX_p)          
Ytr = torch.from_numpy(y_train.reshape(-1,1))  

Xva = torch.from_numpy(ValX_p)                   
Yva = torch.from_numpy(y_val.reshape(-1,1))      

Xte = torch.from_numpy(TestX_p)                  
Yte = torch.from_numpy(y_test.reshape(-1,1))   

pin = (device.type == "cuda")
train_loader = DataLoader(TensorDataset(Xtr, Ytr), batch_size=16, shuffle=True,
                          pin_memory=pin, num_workers=0)
val_loader   = DataLoader(TensorDataset(Xva, Yva), batch_size=64, shuffle=False,
                          pin_memory=pin, num_workers=0)
test_loader  = DataLoader(TensorDataset(Xte, Yte), batch_size=64, shuffle=False,
                          pin_memory=pin, num_workers=0)

print("Hazır: train/val/test loader (CPU → GPU transfer DataLoader içinde yapılacak)")


Hazır: train/val/test loader (CPU → GPU transfer DataLoader içinde yapılacak)


In [6]:

n_qubits = 4
n_layers = 6

dev = qml.device(
    "lightning.gpu",
    wires=n_qubits,
    shots=None,            
    c_dtype=np.complex128, 
)

def ROT_layer(w, n):
    for i in range(n):
        qml.Rot(*w[i], wires=i)

def U3_layer(w, n):
    for i in range(n):
        qml.U3(*w[i], wires=i)

def strong_entangling_layer(n):
    for i in range(n-1):
        qml.CNOT(wires=[i, i+1])
    qml.CNOT(wires=[n-1, 0])

print("Lightning GPU device hazır.")


Lightning GPU device hazır.


In [7]:

@qml.qnode(dev, interface="torch", diff_method="adjoint")
def qnode(inputs, weights_1, weights_2):
    
    qml.AmplitudeEmbedding(features=inputs, wires=range(n_qubits), normalize=True)
    for k in range(n_layers):
        U3_layer(weights_1[k], n_qubits)
        strong_entangling_layer(n_qubits)
        ROT_layer(weights_2[k], n_qubits)
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

weight_shapes = {
    "weights_1": (n_layers, n_qubits, 3),
    "weights_2": (n_layers, n_qubits, 3),
}

qlayer = qml.qnn.TorchLayer(qnode, weight_shapes)


In [8]:

class VQNNModel(nn.Module):
    def __init__(self, nq):
        super().__init__()
        self.q = qlayer           
        self.fc = nn.Linear(2*nq, 1)  
    def forward(self, x):
        x1 = self.q(x)
        x2 = self.q(x**2)
        x  = torch.cat([x1, x2], dim=1)
        x  = F.relu(x)
        return self.fc(x)

model = VQNNModel(n_qubits).to(device)
print(model)


VQNNModel(
  (q): <Quantum Torch Layer: func=qnode>
  (fc): Linear(in_features=8, out_features=1, bias=True)
)


In [9]:

import time

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn   = nn.L1Loss()  # MAE
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min", factor=0.5, patience=5, min_lr=1e-5
)

def run_epoch(loader, train=True):
    if train:
        model.train(True)
    else:
        model.train(False)

    total, count = 0.0, 0
    if train:
        for xb, yb in loader:
            if device.type == "cuda":
                xb = xb.to(device, non_blocking=True)
                yb = yb.to(device, non_blocking=True)
            optimizer.zero_grad(set_to_none=True)
            preds = model(xb)
            loss  = loss_fn(preds, yb)
            loss.backward()
            optimizer.step()
            total += loss.item() * xb.size(0)
            count += xb.size(0)
    else:
        with torch.no_grad():
            for xb, yb in loader:
                if device.type == "cuda":
                    xb = xb.to(device, non_blocking=True)
                    yb = yb.to(device, non_blocking=True)
                preds = model(xb)
                loss  = loss_fn(preds, yb)
                total += loss.item() * xb.size(0)
                count += xb.size(0)
    return total / max(count, 1)

EPOCHS   = 100
PATIENCE = 10
best_val = float("inf")
best_ep  = -1
no_impr  = 0
best_state = None

for ep in range(1, EPOCHS + 1):
    start_t = time.time()

    tr = run_epoch(train_loader, train=True)
    va = run_epoch(val_loader,   train=False)

    scheduler.step(va)
    last_lr = scheduler._last_lr[0] if hasattr(scheduler, "_last_lr") else optimizer.param_groups[0]["lr"]

    # en iyi model kaydı
    if va < best_val - 1e-9:
        best_val  = va
        best_ep   = ep
        no_impr   = 0
        best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
    else:
        no_impr += 1

    elapsed = time.time() - start_t

    if device.type == "cuda":
        torch.cuda.synchronize()
        mem_mb = torch.cuda.memory_allocated() / (1024**2)
        print(f"[Epoch {ep:03d}] Train MAE={tr:.6f} | Val MAE={va:.6f} "
              f"| Best={best_val:.6f}@{best_ep:03d} | Time={elapsed:.2f}s "
              f"| GPU~{mem_mb:.1f}MB | LR={last_lr:.2e}")
    else:
        print(f"[Epoch {ep:03d}] Train MAE={tr:.6f} | Val MAE={va:.6f} "
              f"| Best={best_val:.6f}@{best_ep:03d} | Time={elapsed:.2f}s "
              f"| LR={last_lr:.2e}")

    if no_impr >= PATIENCE:
        print(f"⛳ Early stopping: {PATIENCE} epoch iyileşme yok (son en iyi {best_ep}. epoch).")
        break

if best_state is not None:
    model.load_state_dict(best_state)
    model.to(device).eval()
    print(f"✓ En iyi val MAE = {best_val:.6f} @ epoch {best_ep}")
else:
    print("Uyarı: hiç iyileşme kaydedilmedi.")


[Epoch 001] Train MAE=3.185502 | Val MAE=4.014114 | Best=4.014114@001 | Time=1248.04s | GPU~16.3MB | LR=1.00e-03
[Epoch 002] Train MAE=2.823982 | Val MAE=3.735874 | Best=3.735874@002 | Time=1820.39s | GPU~16.3MB | LR=1.00e-03
[Epoch 003] Train MAE=2.566554 | Val MAE=3.501042 | Best=3.501042@003 | Time=1793.07s | GPU~16.3MB | LR=1.00e-03
[Epoch 004] Train MAE=2.361026 | Val MAE=3.300656 | Best=3.300656@004 | Time=1799.10s | GPU~16.3MB | LR=1.00e-03
[Epoch 005] Train MAE=2.210426 | Val MAE=3.128414 | Best=3.128414@005 | Time=1900.42s | GPU~16.3MB | LR=1.00e-03
[Epoch 006] Train MAE=2.105864 | Val MAE=2.979765 | Best=2.979765@006 | Time=1819.59s | GPU~16.3MB | LR=1.00e-03
[Epoch 007] Train MAE=2.025911 | Val MAE=2.836059 | Best=2.836059@007 | Time=1823.28s | GPU~16.3MB | LR=1.00e-03
[Epoch 008] Train MAE=1.962833 | Val MAE=2.726708 | Best=2.726708@008 | Time=1598.31s | GPU~16.3MB | LR=1.00e-03
[Epoch 009] Train MAE=1.907195 | Val MAE=2.622002 | Best=2.622002@009 | Time=1621.98s | GPU~16.3

KeyboardInterrupt: 

In [10]:

assert best_state is not None, "best_state boş; eğitim boyunca hiç iyileşme olmamış olabilir."

model.load_state_dict(best_state) 
model.eval()
torch.save(model.state_dict(), "best_ampmodel_weights.pth")
print("✓ Gerçek en iyi ağırlıklar kaydedildi: best_ampmodel_weights.pth")


✓ Gerçek en iyi ağırlıklar kaydedildi: best_ampmodel_weights.pth


In [11]:

y_pred, y_true = [], []
with torch.no_grad():
    for xb, yb in test_loader:
        if device.type == "cuda":
            xb = xb.to(device, non_blocking=True)
            yb = yb.to(device, non_blocking=True)
        y_pred.append(model(xb).cpu().numpy().ravel())
        y_true.append(yb.cpu().numpy().ravel())

import numpy as np
from sklearn import metrics
y_pred = np.concatenate(y_pred); y_true = np.concatenate(y_true)

print("Best Test MAE:", metrics.mean_absolute_error(y_true, y_pred))
print("Best Test MSE:", metrics.mean_squared_error(y_true, y_pred))
print("Best Test R^2:",  metrics.r2_score(y_true, y_pred))


Best Test MAE: 1.612947630521802
Best Test MSE: 5.661935281993452
Best Test R^2: 0.5716822834942471


In [ ]:
#model.load_state_dict(torch.load("best_ampmodel_weights.pth", map_location=device))
#model.to(device).eval()


In [20]:
try:
    feature_cols = FEATURES         
except NameError:
    feature_cols = ["BW", "COMED", "DOSE", "TIME", "EVID", "MDV", "AMT", "CMT", "DVID"]

print("Kullanılan kolonlar:", feature_cols)

sample = {
    "BW":   70.0,
    "COMED":0.0,
    "DOSE": 1.0,
    "TIME": 488.0,
    "EVID": 0.0,
    "MDV":  0.0,
    "AMT":  0.0,
    "CMT":  3.0,
    "DVID": 2.0,
}

sample_df = pd.DataFrame([sample], columns=feature_cols).astype(np.float64)

eps_local = 1e-8 if 'eps' not in globals() else float(eps)
x = sample_df.to_numpy(dtype=np.float64)                      
x_std = (x - mean_.astype(np.float64)) / (std_.astype(np.float64) + eps_local)

AMP_DIM = 16
if x_std.shape[1] > AMP_DIM:
    raise ValueError(f"Giriş {x_std.shape[1]} > {AMP_DIM}. Amplitude için 2^n boyuta indirmen gerek.")
pad = np.zeros((1, AMP_DIM - x_std.shape[1]), dtype=np.float64)
x_amp = np.concatenate([x_std, pad], axis=1)                   

xb = torch.from_numpy(x_amp)
if device.type == "cuda":
    xb = xb.to(device, non_blocking=True)

model.eval()
with torch.no_grad():
    y_hat = model(xb).item()

print(f"Tahmin DV (Amplitude, 4 qubit): {y_hat:.6f}")

Kullanılan kolonlar: ['BW', 'COMED', 'DOSE', 'TIME', 'EVID', 'MDV', 'AMT', 'CMT', 'DVID']
Tahmin DV (Amplitude, 4 qubit): 6.384136
